In [14]:
%pip install --upgrade jupyter ipywidgets

   ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
   ----------------- ---------------------- 61.4/139.4 kB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 139.4/139.4 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/215.0 kB ? eta -:--:--
   ----------------------------------- --- 194.6/215.0 kB 12.3 MB/s eta 0:00:01
   ---------------------------------------- 215.0/215.0 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.3 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.3 MB 7.8 MB/s eta 0:00:01
   ----------------------- ---------------- 1.3/2.3 MB 7.7 MB/s eta 0:00:01
   -------------------------------- ------- 1.9/2.3 MB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 9.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/257.4 kB ? eta -:--:--
   -----------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [2]:

import numpy as np
import gc 
import pandas as pd
import optuna
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.ensemble import VotingClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import random
random.seed(42)
np.random.seed(42)

In [3]:
target_col = "churn"

In [4]:
def get_features(df):
    df = df.drop(['slctn_nmbr','postal_code', 'phone_number', 'email'], axis=1)
    df['region'] = LabelEncoder().fit_transform(df["region"])
    df = df.drop_duplicates()
    df['id_'] = df['quarter'] + df['npo_account_id']
    df = df.drop_duplicates("id_")
    
    df['lst_pmnt_date_per_qrtr'] = pd.to_datetime(df['lst_pmnt_date_per_qrtr'])
    df['lst_pmnt_date_per_qrtr_day'] = df['lst_pmnt_date_per_qrtr'].dt.day
    df['lst_pmnt_date_per_qrtr_month'] = df['lst_pmnt_date_per_qrtr'].dt.month
    df['lst_pmnt_date_per_qrtr_year'] = df['lst_pmnt_date_per_qrtr'].dt.year
    df['day_sum1'] = df['lst_pmnt_date_per_qrtr_month']*30+df['lst_pmnt_date_per_qrtr_day']
    df['quarter1'] = LabelEncoder().fit_transform(df["quarter"].copy())
    
    col = ['balance', 'oprtn_sum_per_qrtr', 'oprtn_sum_per_year', 'pmnts_sum', 'incm_per_year',
    'pmnts_nmbr', 'pmnts_sum_per_qrtr', 'pmnts_sum_per_year', 'pmnts_nmbr_per_qrtr', 'pmnts_nmbr_per_year', 'incm_sum', 'incm_per_qrtr']

    for c in tqdm(col):
        data1 = df.groupby('npo_account_id')[c].mean().reset_index()
        data1.columns = ['npo_account_id', f'{c}_quarter']
        df = pd.merge(df, data1, on='npo_account_id', how='left')
        
    data1 = df.groupby('npo_account_id')["quarter"].nunique().reset_index()
    data1.columns = ['npo_account_id', f'quarter_size']
    df = pd.merge(df, data1, on='npo_account_id', how='left')
    df['quartal'] = df['quarter'].str[-1].astype('int32')
    return df

In [5]:
df = pd.read_parquet("C:/Users/Andrey/Documents/Ml/train.parquet")
df = get_features(df)

100%|██████████| 12/12 [00:45<00:00,  3.80s/it]


In [6]:
X = df.drop(columns=[target_col, 'id_', 'client_id', 'npo_account_id', 'quarter',
    'lst_pmnt_date_per_qrtr', 'frst_pmnt_date', 'quarter']).values 
y = df[target_col].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [7]:
'''def objective(trial):
    params = {
        'device':'gpu',
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': 1,
        'boosting_type': 'gbdt',
        'max_depth': trial.suggest_int('max_depth', 6, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-4, 1),
        'n_estimators': 50,
        'num_leaves ': trial.suggest_int('num_leaves', 3, 100)
    }
    model = LGBMClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)
print(study.best_params)
print(study.best_value)
'''

"def objective(trial):\n    params = {\n        'device':'gpu',\n        'objective': 'binary',\n        'metric': 'binary_logloss',\n        'verbosity': 1,\n        'boosting_type': 'gbdt',\n        'max_depth': trial.suggest_int('max_depth', 6, 20),\n        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),\n        'min_child_weight': trial.suggest_float('min_child_weight', 1e-4, 1),\n        'n_estimators': 50,\n        'num_leaves ': trial.suggest_int('num_leaves', 3, 100)\n    }\n    model = LGBMClassifier(**params)\n    model.fit(X_train, y_train)\n    y_pred = model.predict(X_test)\n    score = accuracy_score(y_test, y_pred)\n    return score\n\nstudy = optuna.create_study(direction='maximize')\nstudy.optimize(objective, n_trials=200)\nprint(study.best_params)\nprint(study.best_value)\n"

In [8]:
score_list_acc = []
score_list_f1 = []
strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_index, valid_index in strat_kfold.split(X, y): 
    X_train, X_val = X[train_index], X[valid_index]
    y_train, y_val = y[train_index], y[valid_index]
    
    # 1500
    lgb_params = {'max_depth': 18, 'min_child_samples': 96, 'min_child_weight': 0.13310829983031214, 'num_leaves': 85,
                  'n_estimators':2350, 'device':'gpu', 'boosting_type': 'gbdt'}
    lgb = LGBMClassifier(**lgb_params)
    cat = CatBoostClassifier(iterations=5000, verbose=100, random_state=42, task_type="GPU", devices='0')#3000
    model = VotingClassifier(estimators=[('catboost', cat), ('lgbm', lgb)], voting='soft')
    model.fit(X_train, y_train)
    predict = model.predict(X_val)
    
    score_list_f1.append(f1_score(y_val, predict))
    score_list_acc.append(accuracy_score(y_val, np.round(predict)))
    
    print('f1_score',f1_score(y_val, predict))
    print('accuracy_score', accuracy_score(y_val, np.round(predict)))
    print()
    break
    
print('f1_score', np.mean(score_list_f1))
print('accuracy_score', np.mean(score_list_acc))

# iter = 1200
#roc_auc_score 0.9967203256838154
#accuracy_score 0.9953078990898689

# iter = 1500
#roc_auc_score 0.9969534858449924
#accuracy_score 0.9954274661400886

#roc_auc_score 0.9984288345924386
#accuracy_score 0.9963405236144924

#quartal+
#roc_auc_score 0.9984746575073541
#accuracy_score 0.9963795431646806

#roc_auc_score 0.9984746575073541
#accuracy_score 0.9963795431646806

#ensemble+
#roc_auc_score 0.9981023210843869
#accuracy_score 0.9967850677755651

#roc_auc_score 0.9981023210843869
#accuracy_score 0.9967850677755651

Learning rate set to 0.005144
0:	learn: 0.6792427	total: 283ms	remaining: 23m 34s
100:	learn: 0.1093245	total: 6.31s	remaining: 5m 6s
200:	learn: 0.0499232	total: 12.1s	remaining: 4m 49s
300:	learn: 0.0370382	total: 18.1s	remaining: 4m 42s
400:	learn: 0.0322372	total: 24s	remaining: 4m 35s
500:	learn: 0.0296422	total: 30.1s	remaining: 4m 30s
600:	learn: 0.0279792	total: 36s	remaining: 4m 23s
700:	learn: 0.0268258	total: 41.8s	remaining: 4m 16s
800:	learn: 0.0257960	total: 47.6s	remaining: 4m 9s
900:	learn: 0.0248215	total: 53.6s	remaining: 4m 3s
1000:	learn: 0.0240693	total: 59.5s	remaining: 3m 57s
1100:	learn: 0.0234044	total: 1m 5s	remaining: 3m 51s
1200:	learn: 0.0228247	total: 1m 11s	remaining: 3m 44s
1300:	learn: 0.0222773	total: 1m 16s	remaining: 3m 38s
1400:	learn: 0.0217867	total: 1m 22s	remaining: 3m 32s
1500:	learn: 0.0213496	total: 1m 28s	remaining: 3m 26s
1600:	learn: 0.0209247	total: 1m 34s	remaining: 3m 20s
1700:	learn: 0.0205534	total: 1m 40s	remaining: 3m 14s
1800:	lear

In [ ]:
test_df.head()

,client_id,npo_account_id,npo_accnts_nmbr,pmnts_type,year,quarter,gender,age,clnt_cprtn_time_d,actv_prd_d,...,incm_per_year_quarter,pmnts_nmbr_quarter,pmnts_sum_per_qrtr_quarter,pmnts_sum_per_year_quarter,pmnts_nmbr_per_qrtr_quarter,pmnts_nmbr_per_year_quarter,incm_sum_quarter,incm_per_qrtr_quarter,quarter_size,quartal
0,0x589BC912CB844D468212ADD42EAE4151,0xAA9E8DDF1310724995598EA2B42D7D87,2,1,2017,2017Q2,1,54,6595,4543,...,13188.133846,92.923077,8222.337692,31811.219231,3.0,11.923077,53760.889231,2785.883846,13,2
1,0xCCF22420C90AE4459A25781A9A30F445,0x3B9B09857D152F468A42C5DEE6D723F7,1,3,2021,2021Q4,1,75,7078,0,...,46.570000,1.000000,41.240000,41.240000,0.5,0.500000,110.680000,46.570000,2,4
2,0x943D2C768A4FB38311E6605EEA2B9E4A,0xA7F12C768A4FB38311E835E4A7632E00,1,2,2015,2015Q3,1,35,1675,1615,...,1269.500000,39.000000,1611.650000,6102.815000,3.0,11.000000,1276.225000,0.000000,2,3
3,0x99E64FD29CC3B04189C5C9EBEAF6F9A4,0xBFE4BDFBE0E037478C29BA4F38121B15,1,1,2015,2015Q3,1,50,5180,3538,...,1530.811250,107.125000,945.118750,3699.182500,3.0,11.875000,14920.111250,711.286250,8,3
4,0x654EEEBA731C4A4881816631229C5B65,0x4B5C7804E403C842B4ECBF69BC3EDD7C,2,1,2011,2011Q3,-1,46,296,276,...,4223.670000,66.300000,3617.111000,14736.850000,2.9,12.100000,15609.318000,0.000000,10,3


In [ ]:
test_df.head()

,client_id,npo_account_id,npo_accnts_nmbr,pmnts_type,year,quarter,gender,age,clnt_cprtn_time_d,actv_prd_d,...,incm_per_year_quarter,pmnts_nmbr_quarter,pmnts_sum_per_qrtr_quarter,pmnts_sum_per_year_quarter,pmnts_nmbr_per_qrtr_quarter,pmnts_nmbr_per_year_quarter,incm_sum_quarter,incm_per_qrtr_quarter,quarter_size,quartal
0,0x589BC912CB844D468212ADD42EAE4151,0xAA9E8DDF1310724995598EA2B42D7D87,2,1,2017,2017Q2,1,54,6595,4543,...,13188.133846,92.923077,8222.337692,31811.219231,3.0,11.923077,53760.889231,2785.883846,13,2
1,0xCCF22420C90AE4459A25781A9A30F445,0x3B9B09857D152F468A42C5DEE6D723F7,1,3,2021,2021Q4,1,75,7078,0,...,46.570000,1.000000,41.240000,41.240000,0.5,0.500000,110.680000,46.570000,2,4
2,0x943D2C768A4FB38311E6605EEA2B9E4A,0xA7F12C768A4FB38311E835E4A7632E00,1,2,2015,2015Q3,1,35,1675,1615,...,1269.500000,39.000000,1611.650000,6102.815000,3.0,11.000000,1276.225000,0.000000,2,3
3,0x99E64FD29CC3B04189C5C9EBEAF6F9A4,0xBFE4BDFBE0E037478C29BA4F38121B15,1,1,2015,2015Q3,1,50,5180,3538,...,1530.811250,107.125000,945.118750,3699.182500,3.0,11.875000,14920.111250,711.286250,8,3
4,0x654EEEBA731C4A4881816631229C5B65,0x4B5C7804E403C842B4ECBF69BC3EDD7C,2,1,2011,2011Q3,-1,46,296,276,...,4223.670000,66.300000,3617.111000,14736.850000,2.9,12.100000,15609.318000,0.000000,10,3


In [ ]:
test_df.head()

,client_id,npo_account_id,npo_accnts_nmbr,pmnts_type,year,quarter,gender,age,clnt_cprtn_time_d,actv_prd_d,...,incm_per_year_quarter,pmnts_nmbr_quarter,pmnts_sum_per_qrtr_quarter,pmnts_sum_per_year_quarter,pmnts_nmbr_per_qrtr_quarter,pmnts_nmbr_per_year_quarter,incm_sum_quarter,incm_per_qrtr_quarter,quarter_size,quartal
0,0x589BC912CB844D468212ADD42EAE4151,0xAA9E8DDF1310724995598EA2B42D7D87,2,1,2017,2017Q2,1,54,6595,4543,...,13188.133846,92.923077,8222.337692,31811.219231,3.0,11.923077,53760.889231,2785.883846,13,2
1,0xCCF22420C90AE4459A25781A9A30F445,0x3B9B09857D152F468A42C5DEE6D723F7,1,3,2021,2021Q4,1,75,7078,0,...,46.570000,1.000000,41.240000,41.240000,0.5,0.500000,110.680000,46.570000,2,4
2,0x943D2C768A4FB38311E6605EEA2B9E4A,0xA7F12C768A4FB38311E835E4A7632E00,1,2,2015,2015Q3,1,35,1675,1615,...,1269.500000,39.000000,1611.650000,6102.815000,3.0,11.000000,1276.225000,0.000000,2,3
3,0x99E64FD29CC3B04189C5C9EBEAF6F9A4,0xBFE4BDFBE0E037478C29BA4F38121B15,1,1,2015,2015Q3,1,50,5180,3538,...,1530.811250,107.125000,945.118750,3699.182500,3.0,11.875000,14920.111250,711.286250,8,3
4,0x654EEEBA731C4A4881816631229C5B65,0x4B5C7804E403C842B4ECBF69BC3EDD7C,2,1,2011,2011Q3,-1,46,296,276,...,4223.670000,66.300000,3617.111000,14736.850000,2.9,12.100000,15609.318000,0.000000,10,3


In [32]:
test_df = pd.read_csv("test.csv")

In [33]:
test_df = get_features(test_df)
test_x =test_df.drop(columns=['id_', 'client_id', 'npo_account_id', 'quarter',
    'lst_pmnt_date_per_qrtr', 'frst_pmnt_date', 'quarter']).values 

100%|██████████| 12/12 [00:10<00:00,  1.17it/s]


In [41]:
test_df.head()

,client_id,npo_account_id,npo_accnts_nmbr,pmnts_type,year,quarter,gender,age,clnt_cprtn_time_d,actv_prd_d,...,incm_per_year_quarter,pmnts_nmbr_quarter,pmnts_sum_per_qrtr_quarter,pmnts_sum_per_year_quarter,pmnts_nmbr_per_qrtr_quarter,pmnts_nmbr_per_year_quarter,incm_sum_quarter,incm_per_qrtr_quarter,quarter_size,quartal
0,0x589BC912CB844D468212ADD42EAE4151,0xAA9E8DDF1310724995598EA2B42D7D87,2,1,2017,2017Q2,1,54,6595,4543,...,13188.133846,92.923077,8222.337692,31811.219231,3.0,11.923077,53760.889231,2785.883846,13,2
1,0xCCF22420C90AE4459A25781A9A30F445,0x3B9B09857D152F468A42C5DEE6D723F7,1,3,2021,2021Q4,1,75,7078,0,...,46.570000,1.000000,41.240000,41.240000,0.5,0.500000,110.680000,46.570000,2,4
2,0x943D2C768A4FB38311E6605EEA2B9E4A,0xA7F12C768A4FB38311E835E4A7632E00,1,2,2015,2015Q3,1,35,1675,1615,...,1269.500000,39.000000,1611.650000,6102.815000,3.0,11.000000,1276.225000,0.000000,2,3
3,0x99E64FD29CC3B04189C5C9EBEAF6F9A4,0xBFE4BDFBE0E037478C29BA4F38121B15,1,1,2015,2015Q3,1,50,5180,3538,...,1530.811250,107.125000,945.118750,3699.182500,3.0,11.875000,14920.111250,711.286250,8,3
4,0x654EEEBA731C4A4881816631229C5B65,0x4B5C7804E403C842B4ECBF69BC3EDD7C,2,1,2011,2011Q3,-1,46,296,276,...,4223.670000,66.300000,3617.111000,14736.850000,2.9,12.100000,15609.318000,0.000000,10,3


In [39]:
predict = round(pd.DataFrame(model.predict(test_x)))
predidct_dataframe = test_df[["npo_account_id", "quarter"]]
predidct_dataframe['churn'] = predict
predidct_dataframe.to_csv("predict.csv")

C:\Users\Andrey\AppData\Local\Temp\ipykernel_3820\658742357.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predidct_dataframe['predict'] = predict


In [ ]:
def predict_one_npo_account_id(test_data, npo_account_id, impo=False):
    """
    Возращает для всех определенного  пользователя вероятности
     impo возращать веротяности или False=нет
    """
    col = ['npo_accnts_nmbr', 'pmnts_type', 'year', 'gender', 'age',
       'clnt_cprtn_time_d', 'actv_prd_d', 'lst_pmnt_rcnc_d', 'balance',
       'oprtn_sum_per_qrtr', 'oprtn_sum_per_year', 'frst_pmnt', 'lst_pmnt',
       'pmnts_sum', 'pmnts_nmbr', 'pmnts_sum_per_qrtr', 'pmnts_sum_per_year',
       'pmnts_nmbr_per_qrtr', 'pmnts_nmbr_per_year', 'incm_sum',
       'incm_per_qrtr', 'incm_per_year', 'mgd_accum_period',
       'mgd_payment_period', 'lk', 'assignee_npo', 'assignee_ops', 'region',
       'citizen', 'fact_addrss', 'appl_mrkr', 'evry_qrtr_pmnt',
       'lst_pmnt_date_per_qrtr_day', 'lst_pmnt_date_per_qrtr_month',
       'lst_pmnt_date_per_qrtr_year', 'day_sum1', 'quarter1',
       'balance_quarter', 'oprtn_sum_per_qrtr_quarter',
       'oprtn_sum_per_year_quarter', 'pmnts_sum_quarter',
       'incm_per_year_quarter', 'pmnts_nmbr_quarter',
       'pmnts_sum_per_qrtr_quarter', 'pmnts_sum_per_year_quarter',
       'pmnts_nmbr_per_qrtr_quarter', 'pmnts_nmbr_per_year_quarter',
       'incm_sum_quarter', 'incm_per_qrtr_quarter', 'quarter_size']
    
    test_data = test_data.loc[(test_data.npo_account_id==npo_account_id)]
    
    if impo:
        predict = model.predict_proba(test_data[col].values)[:, 1]
    else:
        predict = model.predict(test_data[col].values)
        
    predidct_dataframe = test_data[["npo_account_id", "quarter"]]
    predidct_dataframe['churn'] = predict
    return predidct_dataframe

In [ ]:
predidct_dataframe.loc[predidct_dataframe.npo_account_id=="0x9DADF88CB3407C4E89403315F640393E"]

In [ ]:
import seaborn as sns
sns.lineplot(predidct_dataframe.loc[predidct_dataframe.npo_account_id=="0x9DADF88CB3407C4E89403315F640393E"].predict)